<a href="https://colab.research.google.com/github/PyLink88/Recurrent-Autoencoder/blob/hyper_tuning/notebooks/Example_Rec_AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Installing required dependencies**

Before install dependencies, recall to set the "Runtime"

In [ ]:
# Installing the necessary dependencies before beginning 

print("Setting up colab environment")
!pip uninstall -y -q pyarrow
!pip install -q -U ray[tune]
!pip install -q ray[debug]

# # A hack to force the runtime to restart, needed to include the above dependencies.
print("Done installing! Restarting via forced crash.")
import os
os._exit(0)

Setting up colab environment
     |████████████████████████████████| 49.7MB 61kB/s 
     |████████████████████████████████| 3.1MB 42.8MB/s 
     |████████████████████████████████| 133kB 2.3MB/s 
     |████████████████████████████████| 81kB 12.0MB/s 
     |████████████████████████████████| 1.3MB 38.6MB/s 
     |████████████████████████████████| 1.0MB 45.3MB/s 
     |████████████████████████████████| 81kB 10.1MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
     |████████████████████████████████| 122kB 61.5MB/s 
     |████████████████████████████████| 143kB 56.7MB/s 
     |████████████████████████████████| 296kB 49.5MB/s 
     |████████████████████████████████| 92kB 12.3MB/s 


# **# Setting the path (IMPORTANT!!!)**

In [2]:
# Setting the path
import os
os.chdir('/content/drive/MyDrive/RecAE')

# **Data downloading and partitioning**


In [3]:
!python utils/data_preparation.py 1 0.5 0.5 0.05

Create ECG5000 folder
### Starting downloading ECG5000 data ###
### Download done! ###
Extracting all the files now...
Extraction done!
Create ECG5000/numpy folder
Saved data in numpy
Data preparation done!


# **Model training and grid search**

For model training with grid search, you can run the following cell, alternatively you can open the file "main_tune.py", set your configuration and then running 


```
!python main_tune.py
```






In [ ]:
import numpy as np
import pandas as pd
import os
from ray import tune
from easydict import EasyDict
from functools import partial

import torch
import torch.nn as nn
import torch.optim as optim

from agents.rnn_autoencoder import RecurrentAEAgent
from graphs.models.recurrent_autoencoder import RecurrentAE
from datasets.common_loader import RecAEDataLoader 

import warnings
warnings.filterwarnings("ignore")

# Project configuration
config_rnn_ae = {

    # Experiment information
    "agent": "RecurrentAEAgent",

    # Architecture hyperparameters
    "rnn_type": "GRU",
    "rnn_act": "None",
    "n_features": 1, # Number of different input signals

    # Optimization hyperparameters
    "batch_size_val": 256,
    "max_epoch": 3,

    # AUC hyperparameters
    'sampler_random_state': 88,

    # Folder where to retrieve the data and their names (IMPORTANT: it must be a global dir)
    "data_folder":  "/content/drive/MyDrive/RecAE/data/ECG5000/numpy/",
    "X_train": "X_train.npy",
    "y_train": "y_train.npy",
    "X_train_p": "X_train_p.npy",
    "y_train_p": "y_train_p.npy",
    "X_val": "X_val.npy",
    "y_val": "y_val.npy",
    "X_test": "X_test.npy",
    "y_test": "y_test.npy",
    "X_val_p": "X_val_p.npy",
    "y_val_p": "y_val_p.npy",

    # GPU settings
    "cuda": True,
    "device": "cuda",
    "gpu_device": 0,
    "seed": 58,

    # Tune
    'tune': True
}

# From dict to easydict
config_rnn_ae = EasyDict(config_rnn_ae)

# First two parameters must be "config" and "checkpoint_dir"
def tune_model(config, checkpoint_dir = None, config_rnn_ae = None):
    
    loss_type, lambda_reg = config['loss_param']
    config['loss_type'] = loss_type
    config['lambda_reg'] = lambda_reg

    # TO DO add the following to directly to the agent class 
    if config["loss_type"] == 'MAE':
      config_rnn_ae.training_type = 'one_class'
    else:
      config_rnn_ae.training_type = 'more_class'

    # Create an instance of the agent
    agent = RecurrentAEAgent(config_rnn_ae)

    # Create an instance from the data loader
    agent.data_loader = RecAEDataLoader(config["batch_size"], agent.config) 

    # Setting the model
    agent.model = RecurrentAE(config["latent_dim"], agent.config)
    agent.model.to(agent.device)

    # Setting the loss
    agent.loss = agent.possible_loss[config["loss_type"]]
    agent.loss.to(agent.device)

    # Setting the optimizer
    agent.optimizer = torch.optim.Adam(agent.model.parameters(), lr = config["lr"])
    agent.train_tune(config['lambda_reg'], checkpoint_dir)

    # Finalizing
    agent.finalize_tune(checkpoint_dir)   
    perf = agent.best_valid

    # Metric to be reported by tune
    tune.report(mean_accuracy = perf)


# Folder where to save experiments the results
my_dir = "/content/drive/MyDrive/RecAE/experiments"

# Project name
project_name ='ECG_5000' # Give a name like the dataset

# Creating nested conditional grid 
def _iter_loss():
    for loss in ['MAE','MAEAUC']:
        if loss == 'MAE':
            yield loss, 0
        else:
            for lambda_reg in [0.001,0.01, 0.1, 1, 10]:
                yield loss, lambda_reg

analysis = tune.run(partial(tune_model, config_rnn_ae = config_rnn_ae), 
                config = {"latent_dim": tune.grid_search([35, 70, 105]),
                          "lr": tune.grid_search([0.001, 0.01]),
                          "batch_size": tune.grid_search([256]),
                          "loss_param": tune.grid_search(list(_iter_loss()))}, 
                resources_per_trial = {"cpu": 2, "gpu": 1}, 
                name = project_name, 
                local_dir = my_dir)


# Saving all results
df = analysis.dataframe()
df.to_pickle(my_dir + '/df_analysis.pkl')



                    





/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)
2021-05-25 07:18:10,077	INFO services.py:1269 -- View the Ray dashboard at http://127.0.0.1:8265
2021-05-25 07:18:11,682	WARNING experiment.py:294 -- No name detected on trainable. Using DEFAULT.
2021-05-25 07:18:11,683	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.


Trial name,status,loc,batch_size,latent_dim,loss_param,lr
DEFAULT_5d087_00000,RUNNING,,256,35,"('MAE', 0)",0.001
DEFAULT_5d087_00001,PENDING,,256,70,"('MAE', 0)",0.001
DEFAULT_5d087_00002,PENDING,,256,105,"('MAE', 0)",0.001
DEFAULT_5d087_00003,PENDING,,256,35,"('MAEAUC', 0.001)",0.001
DEFAULT_5d087_00004,PENDING,,256,70,"('MAEAUC', 0.001)",0.001
DEFAULT_5d087_00005,PENDING,,256,105,"('MAEAUC', 0.001)",0.001
DEFAULT_5d087_00006,PENDING,,256,35,"('MAEAUC', 0.01)",0.001
DEFAULT_5d087_00007,PENDING,,256,70,"('MAEAUC', 0.01)",0.001
DEFAULT_5d087_00008,PENDING,,256,105,"('MAEAUC', 0.01)",0.001
DEFAULT_5d087_00009,PENDING,,256,35,"('MAEAUC', 0.1)",0.001


(pid=545) Operation will be on *****GPU-CUDA***** 


Trial name,status,loc,batch_size,latent_dim,loss_param,lr
DEFAULT_5d087_00000,RUNNING,,256,35,"('MAE', 0)",0.001
DEFAULT_5d087_00001,PENDING,,256,70,"('MAE', 0)",0.001
DEFAULT_5d087_00002,PENDING,,256,105,"('MAE', 0)",0.001
DEFAULT_5d087_00003,PENDING,,256,35,"('MAEAUC', 0.001)",0.001
DEFAULT_5d087_00004,PENDING,,256,70,"('MAEAUC', 0.001)",0.001
DEFAULT_5d087_00005,PENDING,,256,105,"('MAEAUC', 0.001)",0.001
DEFAULT_5d087_00006,PENDING,,256,35,"('MAEAUC', 0.01)",0.001
DEFAULT_5d087_00007,PENDING,,256,70,"('MAEAUC', 0.01)",0.001
DEFAULT_5d087_00008,PENDING,,256,105,"('MAEAUC', 0.01)",0.001
DEFAULT_5d087_00009,PENDING,,256,35,"('MAEAUC', 0.1)",0.001


Result for DEFAULT_5d087_00000:
  date: 2021-05-25_07-18-25
  done: false
  experiment_id: 8a90f6d5017e4b0097cc562b1a23b830
  hostname: d76bb801c9dd
  iterations_since_restore: 1
  mean_accuracy: 1.9955949187278748
  node_ip: 172.28.0.2
  pid: 545
  should_checkpoint: true
  time_since_restore: 12.324602842330933
  time_this_iter_s: 12.324602842330933
  time_total_s: 12.324602842330933
  timestamp: 1621927105
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 5d087_00000
  


...

2021-05-25 07:21:16,948	INFO logger.py:690 -- Removed the following hyperparameter values when logging to tensorboard: {'loss_param': ('MAEAUC', 0.01)}


Result for DEFAULT_5d087_00026:
  date: 2021-05-25_07-21-16
  done: true
  experiment_id: db29d1f8f918437593561778fcac106c
  experiment_tag: 26_batch_size=256,latent_dim=105,loss_param=('MAEAUC', 0.01),lr=0.01
  hostname: d76bb801c9dd
  iterations_since_restore: 1
  mean_accuracy: 1.9393603801727295
  node_ip: 172.28.0.2
  pid: 1283
  should_checkpoint: true
  time_since_restore: 4.699335098266602
  time_this_iter_s: 4.699335098266602
  time_total_s: 4.699335098266602
  timestamp: 1621927276
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 5d087_00026
  


(raylet) /usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)


(pid=1311) Operation will be on *****GPU-CUDA***** 


(pid=1311) /usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
(pid=1311)   % (min_groups, self.n_splits)), UserWarning)
(pid=1311) /usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
(pid=1311)   % (min_groups, self.n_splits)), UserWarning)
(pid=1311) /usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
(pid=1311)   % (min_groups, self.n_splits)), UserWarning)
